In [3]:
import os
import opensmile
import pandas as pd
import numpy as np
from natsort import natsorted
import pandas as pd
from pydub import AudioSegment as AS
from ChaFile import *
from difflib import SequenceMatcher as sm
import editdistance as ed
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter
from sentence_transformers import SentenceTransformer as ST 
from statistics import mean
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\EliKurtz\VSCode_files\NLP\Kurtz-NLP\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

dir_healthy_wav = input("enter the (full) path to your directory of healthy WAV files --> ")
dir_healthy_cha = input("enter the (full) path to your directory of healthy CHA files --> ")

dir_AD_wav = input("enter the (full) path to your directory of AD WAV files --> ")
dir_AD_cha = input("enter the (full) path to your directory of AD CHA files --> ")

dir_MCI_wav = input("enter the (full) path to your directory of MCI WAV files --> ")
dir_MCI_cha = input("enter the (full) path to your directory of MCI CHA files --> ")


In [5]:
def checker(wav, cha):
    files_from_dir = os.listdir(wav)
    files_from_dir_2 = os.listdir(cha)
    if len(files_from_dir) != len(files_from_dir_2):
        quit("not equal")

checker(dir_healthy_wav, dir_healthy_cha)
checker(dir_AD_wav, dir_AD_cha)
checker(dir_MCI_wav, dir_MCI_cha)

AD_cha_files = natsorted(os.listdir(dir_AD_cha))
AD_wav_files = natsorted(os.listdir(dir_AD_wav))

MCI_cha_files = natsorted(os.listdir(dir_MCI_cha))
MCI_wav_files = natsorted(os.listdir(dir_MCI_wav))

Healthy_cha_files = natsorted(os.listdir(dir_healthy_cha))
Healthy_wav_files = natsorted(os.listdir(dir_healthy_wav))

In [6]:
keys_1 = ["command_1", "command_2", "command_3", "command_4", "command_5", "command_6", "command_7", "command_8", "command_9", 
"command_10", "command_11", "command_12", "command_13", "command_14", "command_15", "command_16", "command_17", "command_18", "command_19",
"command_20", "command_21", "command_22", "command_23", "command_24", "command_25", "command_26", "command_27", "command_28", "command_29",
"command_30", "command_31", "command_32", "command_33"]

COMMANDS = ["Alexa what is the weather outside ? ",
"Alexa what is today's date ? ", 
"Alexa what time is it ? ", 
"Alexa when is Thanksgiving ? ",
"Alexa how do you bake chocolate chip cookies ? ",
"Alexa what is two times four ? ",
"Alexa how many tablespoons in a cup ? ",
"Alexa how do you spell symptom ? ",
"Alexa play classical music . ",
"Alexa volume eight . ", 
"Alexa play jazz . ",
"Alexa volume six . ",
"Alexa stop . ",
"Alexa remind me to start the laundry tomorrow at two p_m . ",
"Alexa remind me to feed the dog at seven p_m everyday . ",
"Alexa tell me my reminders . ",
"Alexa remember my daughter's birthday is June first . ", 
"Alexa set a timer in five seconds . ",
"Alexa set my alarm for seven a_m tomorrow . ",
"Alexa add oranges and grapes to my shopping list . ",
"Alexa what is in my shopping list ? ",
"Alexa call six zero three six six zero two two zero three . ",
"Alexa hang up . ",
"Alexa find my phone . ",
"Alexa quit . ",
"Alexa turn the bedroom light on . ",
"Alexa turn the bedroom light red . ",
"Alexa change brightness to ten . ",
"Alexa turn off the bedroom light . ",
"Alexa open the kitchen camera . ",
"Alexa hide the kitchen camera . ",
"Alexa play white collar on fire tv . ",
"Alexa pause . ",
"yes . "]


In [7]:
model = ST('bert-base-nli-mean-tokens')
SENTENCE_EMBEDDINGS = model.encode(COMMANDS)
print(cosine_similarity([SENTENCE_EMBEDDINGS[0]], SENTENCE_EMBEDDINGS[1:]))

[[ 0.5244736   0.52722913  0.426787   -0.02579865  0.33543074  0.18405843
   0.47154203  0.30850616  0.40448692  0.41548067  0.3521029   0.5064732
   0.18790857  0.05849116  0.40963894  0.17014556  0.12249035  0.19524246
   0.10286592  0.35855687  0.33232242  0.46843815  0.34122705  0.4639979
   0.23455638  0.10593515  0.23076949  0.3137503   0.23831546  0.306979
   0.17177723  0.50232565  0.3561165 ]]


In [8]:
utty = model.encode("Hello my fellow americans")
print(cosine_similarity([utty], SENTENCE_EMBEDDINGS))

[[0.21612439 0.3032076  0.3347028  0.26670036 0.13138606 0.24697448
  0.1970247  0.3347801  0.27986488 0.3297175  0.23251595 0.34030795
  0.30949613 0.17863022 0.12827775 0.42476854 0.20201096 0.15331063
  0.17108367 0.13375893 0.26335144 0.34131843 0.44310305 0.2943167
  0.2930704  0.20835713 0.26410308 0.2587133  0.10630944 0.20092665
  0.04604571 0.16089454 0.3293733  0.5842651 ]]


In [36]:
def pct_dist(str1, str2):
    sql = sm(None, str1, str2)
    return sql.ratio()


def wrd_dist(utterance):
    dist_list = []
    for com in COMMANDS:
        dist_list.append([com, ed.eval(com, utterance), utterance])
    min_list = []
    for i in dist_list:
        min_list.append(i[1])
    minimum = min(min_list)
    min_index = min_list.index(minimum)
    return min_index, dist_list


def bert(utterance):
    utty = model.encode(utterance)
    sim = cosine_similarity([utty], SENTENCE_EMBEDDINGS)
    sim_list = sim[0].tolist()
    max_sim = max(sim_list)
    # print(max_sim)
    max_index = sim_list.index(max_sim)
    return [COMMANDS[max_index], utterance]
    
    
def categorize(cha_dir, cha_file, ft: list):
    os.chdir(cha_dir)
    cha = ChaFile(cha_file)
    frame = pd.DataFrame()
    lines = cha.getLines()
    participant_utter = []
    master_list = []
    ur_commands = []
    bert_close_commands = []
    bert_master = []
    close_command_list= []
    for data in lines:
        utterance = data["xol"]
        if "[" in utterance:
            ur_commands.append(utterance)
        else:
            participant_utter.append(utterance)
            min_index, dist_list = wrd_dist(utterance)
            master_list.append(dist_list[min_index])
            bert_closest = bert(utterance)
            bert_master.append(bert_closest)

    df = pd.DataFrame(COMMANDS, columns=["Commands"])
    col_1 = []
    col_2 = []
    col_3 = []
    col_bert = []
    for _ in master_list:
        close_command_list.append(_[0])

    for _ in bert_master:
        bert_close_commands.append(_[0])

    num = Counter(close_command_list)

    dct = {}
    for j in close_command_list:
        dct[j] = [i for i, x in enumerate(close_command_list) if x == j]

    dct_bert = {}
    for j in bert_close_commands:
        dct_bert[j] = [i for i, x in enumerate(bert_close_commands) if x == j]
    print(dct_bert)

    for com in COMMANDS:
        if com not in close_command_list and bert_close_commands:
            col_1.append("Missing")
            col_2.append("Missing")
            col_3.append("Missing")
            col_bert.append("Missing")
        else:
            multi_utter = []
            multi_utter_avg = []
            for utter_index in dct[com]:
                multi_utter.append(master_list[utter_index][2])
                multi_utter_avg.append(master_list[utter_index][1])
            index_of_com = close_command_list.index(com) 
            col_1.append(mean(multi_utter_avg))
            col_2.append(multi_utter)

            multi_utter_bert = []
            for utter_index_bert in dct_bert[com]:
                multi_utter_bert.append(bert_master[utter_index_bert][1])
                print(dct_bert[com])
                print(bert_master[utter_index_bert])
            col_bert.append(multi_utter_bert)
            # Something going wrong here...

            attempts = num[com]
            col_3.append(attempts)
    df["Distance"] = col_1
    df["Utterance_regular_classification"] = col_2
    df["# of Attempts"] = col_3
    df["BERT classification"] = col_bert
    print(len(col_bert))
    print(len(col_3))
    num_ur = len(ur_commands)
    # os.chdir("..")
    # with pd.ExcelWriter("output_new.xlsx", mode="a", engine="openpyxl") as writer:
    #     df.to_excel(writer, sheet_name=cha_file)    

        


In [37]:
ad_ft = []
for i in range(len(AD_cha_files)):
    categorize(dir_AD_cha, AD_cha_files[i], ad_ft)

mci_ft = []
for i in range(len(MCI_cha_files)):
    categorize(dir_MCI_cha, MCI_cha_files[i], mci_ft)

healthy_ft = []
for i in range(len(Healthy_cha_files)):
    categorize(dir_healthy_cha, Healthy_cha_files[i], healthy_ft)

{'Alexa what is the weather outside ? ': [0], "Alexa what is today's date ? ": [1], 'Alexa what time is it ? ': [2], 'Alexa when is Thanksgiving ? ': [3], 'Alexa how do you bake chocolate chip cookies ? ': [4], 'Alexa what is two times four ? ': [5], 'Alexa how many tablespoons in a cup ? ': [6], 'Alexa how do you spell symptom ? ': [7, 25], 'Alexa play classical music . ': [8], 'Alexa volume eight . ': [9], 'Alexa stop . ': [10, 13, 22], 'Alexa play jazz . ': [11], 'Alexa volume six . ': [12], 'Alexa remind me to start the laundry tomorrow at two p_m . ': [14], 'Alexa remind me to feed the dog at seven p_m everyday . ': [15], 'Alexa tell me my reminders . ': [16], "Alexa remember my daughter's birthday is June first . ": [17], 'yes . ': [18], 'Alexa set a timer in five seconds . ': [19, 21], 'Alexa hang up . ': [20], 'Alexa set my alarm for seven a_m tomorrow . ': [23], 'Alexa add oranges and grapes to my shopping list . ': [24], 'Alexa what is in my shopping list ? ': [26], 'Alexa ca

KeyError: 'Alexa play jazz . '

In [ ]:
def flatten(ft):
    ft_flat = []
    for f in range(len(ft)):
        if isinstance(ft[f], list):
            new = list(np.concatenate(ft[f]).flat)
            new.append(ft[f+1])
            ft_flat.append(new)
    return ft_flat
        
ad_ft_flat = flatten(ad_ft)
mci_ft_flat = flatten(mci_ft)
healthy_ft_flat = flatten(healthy_ft)

for i in ad_ft_flat:
    print(len(i))
    print(i)


In [ ]:
def labeler(classic, term):
    blank = []
    for i in classic:
        blank.append(term)
    return blank

ad_labels = labeler(AD_cha_files, "AD")
mci_labels = labeler(MCI_cha_files,"MCI")
healthy_labels = labeler(Healthy_cha_files, "Healthy")

labels = ad_labels + mci_labels + healthy_labels
